<a href="https://colab.research.google.com/github/osvaldoguax/pantanal-ph-prediction/blob/master/C%C3%B3pia_de_ee_api_colab_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/ee-api-colab-setup.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/ee-api-colab-setup.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Introduction: Mapping crops in Brazil using simulated Landsat images

This notebook demonstrates how to map crops (soy, cotton and corn) in Brazil based on high spatial and temporal resolution simualted satellite images. The mapping is based on ESTARFM method, which is a temporal linear regression method allowing simulating Landsat bands based on adjacent MODIS images bands. The ESTARFM code was scripted based on the assumptions presented by Knauer et al (2016), see for more details: 
Knauer, K.; Gessner, U.; Fensholt, R.; Kuenzer, C. An ESTARFM Fusion Framework for the Generation of Large-Scale 
Time Series in Cloud-Prone and Heterogeneous Landscapes. Remote Sens. 2016, 8, 425.

## Authenticate and initialize

Run the 'import ee' and the `ee.Authenticate` functions to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=XrnfBDmkQWyPUnMtDv4vgQOWD35u7ImOLEnsG1aynJ0&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/2AHjwdahj2E2QYs08TR_vDih2c-1r3Fh7KHkWLUNwswWv5pW1LYsLoc

Successfully saved authorization token.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Interactive map

The [`folium`](https://python-visualization.github.io/folium/)
library can be used to display `ee.Image` objects on an interactive
[Leaflet](https://leafletjs.com/) map. Folium has no default
method for handling tiles from Earth Engine, so one must be defined
and added to the `folium.Map` module before use.

The following cell provides an example of adding a method for handing Earth Engine
tiles and using it to display an elevation model to a Leaflet map.

In [ ]:
import folium

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

print('Folium version: ' + folium.__version__)

#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

Folium version: 0.8.3


## Implementing editable variables

Here are presented the variables that can be changed according to the user's interest. The study area, the temporal filter for satellite image collections, the mapping period of agricultural areas and the year of the MapBiomas Land Use Land Cover (LULC) map can be modified.

In [ ]:
#Study area. Subset of a small region in Bahia state
polygon = ee.Geometry.Polygon(
        [[[-46.333915055332604, -12.467720172610312],
          [-46.333915055332604, -13.115892713759077],
          [-45.040274918613854, -13.115892713759077],
          [-45.040274918613854, -12.467720172610312]]]);
time ='system:time_start'

#Temporal winwow defined to filter Landsat and MODIS collection (It is necessary to keet this date as it is to take into account the entire Landsat 8 collection)
compositestart = ee.Date('2014-01-01');
compositeend = ee.Date('2020-12-31');

#Convert initial and final dates to milliseconds
startMillis = compositestart.millis();
endMillis = compositeend.millis();

#Data range for crop mapping
crop_initial = ee.Date('2017-12-15');#initial date to map croplands (first day of the year of interest)
crop_final = ee.Date('2018-10-31');#final date to map croplands (first day of the year of interest)
seeding_season = ee.Filter.calendarRange(12, 2, 'month')
resting_season = ee.Filter.calendarRange(8, 9, 'month')
labeled_data = ee.Image('projects/mapbiomas-workspace/public/collection4_1/mapbiomas_collection41_integration_v1').toUint8().clip(polygon).select(['classification_2018'])#Ground truth data used to classify the origibal and simulated Landsat data
CEI_threshold = [170];#Threshold used to filter the data

## Landsat cloud mask 

Cloud mask to remove clouds and cloud shadows in reflectance images at the top of the atmosphere

In [ ]:
def getQABits(image, start, end, mascara): # Compute the bits we need to extract.
    pattern = 0
    for i in range(start,end+1):
        pattern += 2**i
    return image.select([0], [mascara]).bitwiseAnd(pattern).rightShift(start) 

#A function to mask out cloudy pixels.
def maskQuality(image):
    QA = image.select('BQA')
    sombra = getQABits(QA,7,8,'cloud_shadow')
    nubes = getQABits(QA,4,4,'cloud')
    cirrus_detected = getQABits(QA,11,12,'cirrus_detected')
    return image.updateMask(sombra.eq(1)).updateMask(nubes.neq(1).updateMask(cirrus_detected.gt(0))) 

# Load the Landsat 8 image collection, filter to the area of interest and acquisition period, cloud masking
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterDate(compositestart, compositeend).filterBounds(polygon).map(maskQuality)

# Compute the median in each band, each pixel.
# Band names are B1_median, B2_median, etc.
# Median Landsat collection for visualization
median = collection.filterDate(crop_initial,crop_final).reduce(ee.Reducer.median()).clip(polygon)

# The output is an Image.  Add it to the map.
vis_param = {'bands': ['B7_median', 'B5_median', 'B4_median'],
             'min': 0.03,
             'max': 0.6}

median_tk = median.getMapId(vis_param)

# Load an image collection, filtered so it's not too much data.
collection_cloud = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')\
               .filterDate(crop_initial, crop_final).filterBounds(polygon)
median_cloud = collection_cloud.reduce(ee.Reducer.median()).clip(polygon)
median_tk_cloud = median_cloud.getMapId(vis_param)

## Landsat 8 image cube: composition used for the ESTFARM simulation

Final Landsat 8 image cube that will be used for simulation. In this code the NDVI band is created and the images are converted to intergers

In [ ]:
ls0 = collection.select(['B2', 'B4', 'B5', 'B7'], ['ls_B1', 'ls_B3', 'ls_B4', 'ls_B7'])

band_ls = ee.Image(ls0.first()).select('ls_B7')
proj_ls = band_ls.projection().getInfo()
crs_ls = proj_ls['crs']

time ='system:time_start'

ls1 = ls0.map(lambda image:
      image.set('count', image.bandNames().length())
      ).filter(ee.Filter.eq('count', 4)).map(lambda m:
      m.addBands(m.normalizedDifference(['ls_B4', 'ls_B3']).rename('ls_NDVI'))).map(lambda m:
      m.reproject(crs=crs_ls, scale = 30).resample('bicubic').reproject(crs=crs_ls, scale = 100).multiply(10000).toUint16().copyProperties(m,[time]));

## MODIS 500m image cube: composition used for the ESTFARM simulation

Prepare the collection and resample to 100m to be able to make the simulation based on MODIS data

In [ ]:
#MODIS surface reflectance
def getQABits(image, start, end, mascara):
    pattern = 0
    for i in range(start,end+1):
        pattern += 2**i
    return image.select([0], [mascara]).bitwiseAnd(pattern).rightShift(start)

#A function to mask out cloudy pixels.
def maskQuality(image):
    QA = image.select('SummaryQA')
    internalQuality = getQABits(QA,0, 1, 'internal_quality_flag')
    return image.updateMask(internalQuality.lt(2))

#Load MODIS Collection
modsr = ee.ImageCollection('MODIS/006/MOD13A1').map(maskQuality).filterDate(startMillis,endMillis).filterBounds(polygon)

#Select the first MODIS image to look up the projection
band_proj = ee.Image(modsr.first()).select('sur_refl_b07')
proj = band_proj.projection().getInfo()
crs = proj['crs']

#MODIS collection for simulation
MODIS = modsr.map(lambda img : img.select(['sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b07', 'NDVI'], ['B3_mod', 'B4_mod', 'B1_mod', 'B7_mod', 'NDVI_mod'])
.reproject(crs=crs, scale=500).resample('bicubic').reproject(crs=crs, scale=100)
.copyProperties(img,[time])).filterDate(compositestart, compositeend).filterBounds(polygon);

## Adding the created composition to the map: Landsat and MODIS

Declaration of variables to present the image compositions that will be used for the simulation

In [ ]:
#Median MODIS for visualization
median_modis = MODIS.filterDate(crop_initial,crop_final).reduce(ee.Reducer.median()).clip(polygon)

#The output is an Image.  Add it to the map.
vis_param_m = {'bands': ['B7_mod_median', 'B4_mod_median', 'B3_mod_median'],
             'min': 300,
             'max': 6000}

median_modis_tk = median_modis.getMapId(vis_param_m)

center = [-12.7759,-45.694]
Mapdisplay(center,{'MODIS':median_modis_tk, 'Landsat 8 sem nuvem':median_tk, 'Landsat 8 com nuvem':median_tk_cloud},zoom_start=11)

# Creating the ESTARFM filter parameters to simulated Landsat images based on MODIS data

In [ ]:
#The ESTARFM code was scripted based on the assumptions presented by Knauer et al (2016), see for more details: 
#Knauer, K.; Gessner, U.; Fensholt, R.; Kuenzer, C. An ESTARFM Fusion Framework for the Generation of Large-Scale 
#Time Series in Cloud-Prone and Heterogeneous Landscapes. Remote Sens. 2016, 8, 425.

#Join Landsat and MODIS closest scenes for the ESTARFM regression
#Raster Join

# Define an allowable time difference: two days in milliseconds.
twoDaysMillis = 24 * 24 * 60 * 60 * 1000 #We are using 24 days at left and 24 days at right. Any Landsat collection within this interval will be accepted to merge with MODIS

# Create a time filter to define a match as overlapping timestamps.
maxDiff = ee.Filter.Or(
ee.Filter.maxDifference(
    difference= twoDaysMillis,
    leftField='system:time_start',
    rightField= 'system:time_start'
  ),
ee.Filter.maxDifference(
    difference=twoDaysMillis,
    leftField='system:time_start',
    rightField='system:time_start'
  )
)

# Define the join.
join=ee.Join.saveAll(
  matchesKey='B',
  ordering='system:time_start',
  measureKey='timeDiff'
)

prior =ee.Filter.greaterThan(leftField=time,rightField=time);
filter1 = ee.Filter.And(maxDiff, prior);
post = ee.Filter.lessThan(leftField=time,rightField=time);
filter2 = ee.Filter.And(maxDiff,post);
filter = ee.Filter.Or(filter1,filter2);

## Here we apply the simulation for each Landsat/MODIS band

###NDVI band

In [ ]:
#Applying joing between MODIS and Landsat 
lsmodis = ee.ImageCollection(join.apply(ls1,MODIS,filter)).map(lambda img:
img.toShort().clip(polygon));

#A function to mask out cloudy pixels.
def NDVI_def(img):
    maxmodis = ee.ImageCollection.fromImages(img.get('B')).select('NDVI_mod').max()
    return maxmodis.addBands(ee.Image(1)).addBands(img.select('ls_NDVI')).select([0, 1, 2],['maxmodis','const1','ls_NDVI']).copyProperties(img,[time])
NDVI = lsmodis.map(NDVI_def)

coeffs_NDVI = NDVI.reduce(ee.Reducer.linearRegression(2,1)).select('coefficients')

slope_NDVI = coeffs_NDVI.arrayGet([0,0])

intercept_NDVI = coeffs_NDVI.arrayGet([1,0])

def Fitted_NDVI_def(img):
  fit1 = img.select(2).add(intercept_NDVI.add(img.select(0).multiply(slope_NDVI))).multiply(0.5)
  fit2 = fit1.unmask(intercept_NDVI.add(img.select(0).multiply(slope_NDVI))).rename('fitted_NDVI')
  return fit2.copyProperties(img,[time]).copyProperties(img,['system:index'])
fitted_NDVI = NDVI.map(Fitted_NDVI_def)

def fitted_NDVIf_def(img):
  result = intercept_NDVI.add(img.select('NDVI_mod').multiply(slope_NDVI)).rename('30m_NDVI')
  return result.copyProperties(img,[time]).copyProperties(img,['system:index'])
fitted_NDVIf = MODIS.select('NDVI_mod').map(fitted_NDVIf_def)

### Band 1 (Blue)

In [ ]:
#Applying joing between MODIS and Landsat 
lsmodis = ee.ImageCollection(join.apply(ls1,MODIS,filter)).map(lambda img:
img.toShort().clip(polygon));

#A function to mask out cloudy pixels.
def B1_def(img):
    maxmodis = ee.ImageCollection.fromImages(img.get('B')).select('B1_mod').max()
    return maxmodis.addBands(ee.Image(1)).addBands(img.select('ls_B1')).select([0, 1, 2],['maxmodis','const1','ls_B1']).copyProperties(img,[time])
B1 = lsmodis.map(B1_def)

coeffs_B1 = B1.reduce(ee.Reducer.linearRegression(2,1)).select('coefficients')

slope_B1 = coeffs_B1.arrayGet([0,0])

intercept_B1 = coeffs_B1.arrayGet([1,0])

def Fitted_B1_def(img):
  fit1 = img.select(2).add(intercept_B1.add(img.select(0).multiply(slope_B1))).multiply(0.5)
  fit2 = fit1.unmask(intercept_B1.add(img.select(0).multiply(slope_B1))).rename('fitted_B1')
  return fit2.copyProperties(img,[time]).copyProperties(img,['system:index'])
fitted_B1 = B1.map(Fitted_B1_def)

def fitted_B1f_def(img):
  result = intercept_B1.add(img.select('B1_mod').multiply(slope_B1)).rename('30m_B1')
  return result.copyProperties(img,[time]).copyProperties(img,['system:index'])
fitted_B1f = MODIS.select('B1_mod').map(fitted_B1f_def)

### Band 3 (Red)

In [ ]:
#Applying joing between MODIS and Landsat 
lsmodis = ee.ImageCollection(join.apply(ls1,MODIS,filter)).map(lambda img:
img.toShort().clip(polygon));

#A function to mask out cloudy pixels.
def B3_def(img):
    maxmodis = ee.ImageCollection.fromImages(img.get('B')).select('B3_mod').max()
    return maxmodis.addBands(ee.Image(1)).addBands(img.select('ls_B3')).select([0, 1, 2],['maxmodis','const1','ls_B3']).copyProperties(img,[time])
B3 = lsmodis.map(B3_def)

coeffs_B3 = B3.reduce(ee.Reducer.linearRegression(2,1)).select('coefficients')

slope_B3 = coeffs_B3.arrayGet([0,0])

intercept_B3 = coeffs_B3.arrayGet([1,0])

def Fitted_B3_def(img):
  fit1 = img.select(2).add(intercept_B3.add(img.select(0).multiply(slope_B3))).multiply(0.5)
  fit2 = fit1.unmask(intercept_B3.add(img.select(0).multiply(slope_B3))).rename('fitted_B3')
  return fit2.copyProperties(img,[time]).copyProperties(img,['system:index'])
fitted_B3 = B3.map(Fitted_B3_def)

def fitted_B3f_def(img):
  result = intercept_B3.add(img.select('B3_mod').multiply(slope_B3)).rename('30m_B3')
  return result.copyProperties(img,[time]).copyProperties(img,['system:index'])
fitted_B3f = MODIS.select('B3_mod').map(fitted_B3f_def)

### Band 4 (NIR)

In [ ]:
#Applying joing between MODIS and Landsat 
lsmodis = ee.ImageCollection(join.apply(ls1,MODIS,filter)).map(lambda img:
img.toShort().clip(polygon));

#A function to mask out cloudy pixels.
def B4_def(img):
    maxmodis = ee.ImageCollection.fromImages(img.get('B')).select('B4_mod').max()
    return maxmodis.addBands(ee.Image(1)).addBands(img.select('ls_B4')).select([0, 1, 2],['maxmodis','const1','ls_B4']).copyProperties(img,[time])
B4 = lsmodis.map(B4_def)

coeffs_B4 = B4.reduce(ee.Reducer.linearRegression(2,1)).select('coefficients')

slope_B4 = coeffs_B4.arrayGet([0,0])

intercept_B4 = coeffs_B4.arrayGet([1,0])

def Fitted_B4_def(img):
  fit1 = img.select(2).add(intercept_B4.add(img.select(0).multiply(slope_B4))).multiply(0.5)
  fit2 = fit1.unmask(intercept_B4.add(img.select(0).multiply(slope_B4))).rename('fitted_B4')
  return fit2.copyProperties(img,[time]).copyProperties(img,['system:index'])
fitted_B4 = B4.map(Fitted_B4_def)

def fitted_B4f_def(img):
  result = intercept_B4.add(img.select('B4_mod').multiply(slope_B4)).rename('30m_B4')
  return result.copyProperties(img,[time]).copyProperties(img,['system:index'])
fitted_B4f = MODIS.select('B4_mod').map(fitted_B4f_def)

### Band 7 (SWIR)

In [ ]:
#Applying joing between MODIS and Landsat 
lsmodis = ee.ImageCollection(join.apply(ls1,MODIS,filter)).map(lambda img:
img.toShort().clip(polygon));

#A function to mask out cloudy pixels.
def B7_def(img):
    maxmodis = ee.ImageCollection.fromImages(img.get('B')).select('B7_mod').max()
    return maxmodis.addBands(ee.Image(1)).addBands(img.select('ls_B7')).select([0, 1, 2],['maxmodis','const1','ls_B7']).copyProperties(img,[time])
B7 = lsmodis.map(B7_def)

coeffs_B7 = B7.reduce(ee.Reducer.linearRegression(2,1)).select('coefficients')

slope_B7 = coeffs_B7.arrayGet([0,0])

intercept_B7 = coeffs_B7.arrayGet([1,0])

def Fitted_B7_def(img):
  fit1 = img.select(2).add(intercept_B7.add(img.select(0).multiply(slope_B7))).multiply(0.5)
  fit2 = fit1.unmask(intercept_B7.add(img.select(0).multiply(slope_B7))).rename('fitted_B7')
  return fit2.copyProperties(img,[time]).copyProperties(img,['system:index'])
fitted_B7 = B7.map(Fitted_B7_def)

def fitted_B7f_def(img):
  result = intercept_B7.add(img.select('B7_mod').multiply(slope_B7)).rename('30m_B7')
  return result.copyProperties(img,[time]).copyProperties(img,['system:index'])
fitted_B7f = MODIS.select('B7_mod').map(fitted_B7f_def)

## Creating the Crop Enhaced Index band for simulated and origibal Landsat

In [ ]:
#--------------------------------------Building CEI (Crop Enhancement Index) index for fitted and original Landsat compositions---------------------------------------------------
#CEI on fitted data
max_Simu = fitted_NDVIf.filterDate(crop_initial,crop_final).filter(seeding_season).max().select('30m_NDVI');
min_Simu = fitted_NDVIf.filterDate(crop_initial,crop_final).filter(resting_season).min().select('30m_NDVI');
cei_2019_MOD = max_Simu.expression('1000 *(WET_max - DRY_min) / ((1000 + WET_max) + (1000 + DRY_min))', {
			'WET_max': max_Simu,
			'DRY_min': min_Simu,
}).rename('CEI_FIT_2018').clip(polygon);

#CEI on fitted data
max_Landsat= ls1.filterDate(crop_initial,crop_final).filter(seeding_season).max().select('ls_NDVI');
min_Landsat = ls1.filterDate(crop_initial,crop_final).filter(resting_season).min().select('ls_NDVI');
cei_2019_LS = max_Landsat.expression('1000 *(WET_max - DRY_min) / ((1000 + WET_max) + (1000 + DRY_min))', {
			'WET_max': max_Landsat,
			'DRY_min': min_Landsat,
}).rename('CEI_LS_2018').clip(polygon);

## Merging simulated bands to create the Landsat/MODIS simulated image for classification
 
 This composition is the product for the crop (soy) classification based on simulated data


In [ ]:
multi_simu = fitted_B7f.combine(fitted_B4f).combine(fitted_B3f).combine(fitted_B1f).combine(fitted_NDVIf).select(
    ['30m_B7', '30m_B4', '30m_B3', '30m_B1', '30m_NDVI'], ['Fit_B7', 'Fit_B4', 'Fit_B3', 'Fit_B1', 'NDVI']
    ).filterDate(crop_initial, crop_final).filter(seeding_season).reduce(ee.Reducer.median()).addBands(cei_2019_MOD)

## Merging original Landsat bands to create the Landsat image for classification

 This composition is the product for the crop (soy) classification based on original Landsat data

In [ ]:
multi_original = ls1.filterDate(crop_initial, crop_final).filter(seeding_season).reduce(ee.Reducer.median()).addBands(cei_2019_LS).clip(polygon)

## View the final result of the Landsat simulation compared to the original Landsat composition

Here we also included the labeled data by [MapBiomas](https://mapbiomas.org/). This data is the groud truth that can be used to verify the map quality and to classify the Landsat simulated and original data. The original GEE asset is: projects/mapbiomas-workspace/public/collection4_1/mapbiomas_collection41_integration_v1

In [ ]:
#The output is an Image.  Add it to the map.
vis_param_multi = {'bands': ['Fit_B7_median', 'Fit_B4_median', 'Fit_B3_median'],
             'min': 300,
             'max': 4500}

#The output is an Image.  Add it to the map.
vis_param_original = {'bands': ['ls_B7_median', 'ls_B4_median', 'ls_B3_median'],
             'min': 300,
             'max': 6000}

#The output is an Image.  Add it to the map.
vis_param_labeled_data = {'palette': ['c0c0c0', 'cc6600'],
             'min': 1,
             'max': 2}

labeled_soja = labeled_data.eq(19).remap([0, 1],[0, 2]) #select the soy category from MapBiomas
labeled_no_soja = labeled_data.neq(19).remap([0, 1],[0, 1])  #select the non-soy categories

#Sum maps
soy_map = labeled_no_soja.add(labeled_soja)#sum data

#Final labeled data for classification
final_labeled_data = soy_map.updateMask(soy_map.gt(0)).clip(polygon).toByte();

median_multi_tk = multi_simu.getMapId(vis_param_multi)
median_original_tk = multi_original.getMapId(vis_param_original)
labeled_data_tk = final_labeled_data.getMapId(vis_param_labeled_data)

center = [-12.7759,-45.694]
Mapdisplay(center,{'2017 soy area: Ground Truth': labeled_data_tk, 'Landsat 8 original':median_original_tk, 'Landsat 8 simulated':median_multi_tk},zoom_start=11)

# Cropland Random Forest classification

The Classification is carried for the year of 2017. That's the date of the labeled data by Agrosatélite 

##Classifying the simulated Landsat image based on MapBiomas ground truth

In [ ]:
points = 1000;
trees = 100;
seeds = 0;

training_simu = multi_simu.addBands(final_labeled_data).sample(
  region=polygon,
  numPixels=1000,
  scale=100,
  seed=0
)

bands_class_simu = ['Fit_B7_median', 'Fit_B4_median', 'Fit_B3_median', 'Fit_B1_median', 'NDVI_median', 'CEI_FIT_2018']

classifier_simu = ee.Classifier.randomForest(trees, seeds).train(
  features = training_simu, 
  classProperty = 'remapped',
  inputProperties = bands_class_simu
)

#Classificar a imagem Sentinel com base em Random Forest.
classified_simu = multi_simu.classify(classifier_simu);

##Classifying the original Landsat image based on MapBiomas ground truth

In [ ]:
training_original = multi_original.addBands(final_labeled_data).sample(
  region=polygon,
  numPixels=1000,
  scale=100,
  seed=0
)

bands_class_original = ['ls_B7_median', 'ls_B4_median', 'ls_B3_median', 'ls_B1_median', 'ls_NDVI_median', 'CEI_LS_2018']

classifier_original = ee.Classifier.randomForest(trees, seeds).train(
  features = training_original, 
  classProperty = 'remapped',
  inputProperties = bands_class_original
)

#Classificar a imagem Sentinel com base em Random Forest.
classified_original = multi_original.classify(classifier_original);

# Final map
This example shows a subset of soy area classification comparing the classification of the simulated Landsat and the original Landsat. The original composition is clearly affected by cloud coverage.

![image hilighting the final result: Example at west Bahia/Brazil](https://www.dropbox.com/s/yrptmm3fesbi4q3/Imagem1.png?raw=1)

In [ ]:
#The output is an Image.  Add it to the map.
vis_param_labeled_data = {'palette': ['c0c0c0', 'cc6600'],
             'min': 1,
             'max': 2}

simu_classified_tk = classified_simu.getMapId(vis_param_labeled_data)
original_classified_tk = classified_original.getMapId(vis_param_labeled_data)

center = [-12.7759,-45.694]
Mapdisplay(center,{'Simulated classified':simu_classified_tk,'Original classified': original_classified_tk, 'Ground Truth': labeled_data_tk},zoom_start=11)